In [1]:
import subprocess
import time
from configurations.configuration import Configuration
from communication.send_data import sendObject
from exp.params import (
    REP_PORT,
    PATH_TO_TASKS, 
    PATH_TO_CONFIG_FILE, 
    SERVER_REPLICA_MANAGER_PORT,
    MEMCACHED_LISTENING_PORT,
)

[WARNING]: failed to patch stdout/stderr for fork-safety: 'OutStream' object
has no attribute 'buffer'
[WARNING]: failed to reconfigure stdout/stderr with the replace error handler:
'OutStream' object has no attribute 'reconfigure'
/Users/cherif/Documents/Traveaux/traces-simulator/venv/lib/python3.12/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/Users/cherif/Documents/Traveaux/traces-simulator/venv/lib/python3.12/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
def run_command(command):
    
    result = subprocess.run(command.split(), capture_output=True, text=True)
    print(result)

def InfosToSend(id_peer:int,graphe_info,ip_address, rep_port, cache_size):
    data = {}
    data["CACHE_SIZE"] = cache_size*1024*1024
    data["infos"] = []
    data["SITE_ID"] = id_peer
    data["REP_PORT"] = rep_port + id_peer
    data["IP_ADDRESS"] = str(ip_address[id_peer])
    data["MANAGER_IP"] = str(ip_address[-1])
    
    for i in range(len(graphe_info)):
        if graphe_info[id_peer,i] > 0:
            peer = {
                'latency' : graphe_info[id_peer,i],
                "id": i,
                "ip" : ip_address[i], 
                "rep_port" : rep_port+i,
            } 

            data["infos"].append(peer)
    return data

def SendInfoToManager(id_peer, config,ips_address, rep_port):
    data = {}
    data["IP_ADDRESS"] = str(ips_address[id_peer])
    data['graphe_infos'] = config
    data["IPs_ADDRESS"] = ips_address
    data["infos"] = {}
    
    for i in range(len(config)):
        if config[id_peer,i] > 0:
            peer = {
                'latency' : config[id_peer,i],
                "id": i,
                "node_ip" : ips_address[i], 
                "node_port" : rep_port+i,
            } 
            data["infos"][i] = peer
    return data

In [3]:
port_rep = REP_PORT



config = Configuration(
    config_file_path = "/Users/cherif/Documents/Traveaux/traces-simulator/cache-exp/configurations/conf.yaml",
    memcached_listening_port=MEMCACHED_LISTENING_PORT
)

provider = config.setReservation()
netem = config.setNetworkConstraintes()

## deplot memcached
config.deployMemcached(port=MEMCACHED_LISTENING_PORT)

NB_NODES = config.nb_sites
CONFIG_GRAPHE = config.getGraphe()
IPS_ADDRESS = config.getAllIPs()

print(IPS_ADDRESS)
#config.provider.destroy()

infos_nodes= []
    
if config.execution_local:
    import threading
    for i, machine in enumerate(config.machines):
        if i == NB_NODES-1:

            data = SendInfoToManager(i, CONFIG_GRAPHE,IPS_ADDRESS, REP_PORT)
            
            thread = threading.Thread(
                target=run_command, 
                args=(f"python replicaManager.py ",))
            
            thread.start()
            
            print(f"adresse IP du node {i} : {IPS_ADDRESS[i]}")
            
            sendObject(data, IPS_ADDRESS[i])
            infos_nodes.append({"node_ip":IPS_ADDRESS[i], "node_port":SERVER_REPLICA_MANAGER_PORT})
            port_rep += 1
            time.sleep(1)
        else:
            data = InfosToSend(i,CONFIG_GRAPHE, IPS_ADDRESS,REP_PORT,config.storage_capacities[i])
            
            thread = threading.Thread(
                target=run_command, 
                args=(f"python cacheManager.py ",))
            
            thread.start()
            
            print(f"adresse IP du node {i} : {IPS_ADDRESS[i]}")
            
            sendObject(data, "localhost")
            infos_nodes.append({"node_ip":IPS_ADDRESS[i], "node_port":port_rep})
            port_rep += 1
            time.sleep(1)
                
        
 
    
        """print("Waiting for Outputs:")
        count = 0
        time.sleep(60)

        while True:
            print(count)
            if count == config.nb_sites:
                break

            for i, machine in enumerate(config.machines): 
                try:
                    with config.enoslib.actions(roles=config.roles[machine["roles"][0]]) as p:
                        p.fetch(src=f"/tmp/log_{i}.txt", dest="~")  
                        #p.fetch(src=f"/tmp/log_{i}.err", dest="~")
                        #p.fetch(src=f"/tmp/log_{i}.out", dest="~")    
                    print("Output fetched")
                    count +=1                    
                except Exception:
                    continue"""
        

_____        ___  ____  _ _ _
 | ____|_ __  / _ \/ ___|| (_) |__
 |  _| | '_ \| | | \___ \| | | '_ \
 | |___| | | | |_| |___) | | | |_) |
 |_____|_| |_|\___/|____/|_|_|_.__/  9.3.0

 • Documentation: ]8;id=588229;https://discovery.gitlabpages.inria.fr/enoslib/\https://discovery.gitlabpages.inria.fr/enoslib/]8;;\                            
 • Source: ]8;id=72253;https://gitlab.inria.fr/discovery/enoslib\https://gitlab.inria.fr/discovery/enoslib]8;;\                                         
 • Chat: ]8;id=84342;https://framateam.org/enoslib\https://framateam.org/enoslib]8;;\

                         Dependency check                         
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Provider      ┃    Status     ┃ Hint                           ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Chameleon     │ NOT INSTALLED │ pip install enoslib[chameleon] │
│ ChameleonKVM  │ NOT INSTALLED │ pip install enoslib[chameleon] │
│ ChameleonEdge │ NOT INSTALLED │ pip install enoslib[chameleon] │
│ Distem        │ NOT INSTALLED │ pip install enoslib[distem]    │
│ IOT-lab       │ NOT INSTALLED │ pip install enoslib[iotlab]    │
│ Grid'5000     │   INSTALLED   │                                │
│ Openstack     │ NOT INSTALLED │ pip install enoslib[chameleon] │
│ Vagrant       │ NOT INSTALLED │ pip install enoslib[vagrant]   │
│ VMonG5k       │   INSTALLED   │                                │
└───────────────┴───────────────┴────────────────────────────────┘

                                Connectivity check                                 
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Provider  ┃ Key                 ┃ Connectivity ┃ Hint                           ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Grid'5000 │ ssh:access          │      ✅      │ Connection to access.grid5000… │
│ Grid'5000 │ ssh:access:frontend │      ✅      │ Connection Host(rennes.grid50… │
│ Grid'5000 │ api:access          │      ✅      │                                │
│ VMonG5k   │ access              │      ❔      │ Check G5k status               │
└───────────┴─────────────────────┴──────────────┴────────────────────────────────┘

INFO     [G5k] Submitting {'name': 'EnOSlib', 'types': [], 'resources':  ]8;id=754869;file:///Users/cherif/Documents/Traveaux/traces-simulator/venv/lib/python3.12/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py\g5k_api_utils.py]8;;\:]8;id=226705;file:///Users/cherif/Documents/Traveaux/traces-simulator/venv/lib/python3.12/site-packages/enoslib/infra/enos_g5k/g5k_api_utils.py#305\305]8;;\
         "{cluster='paravance'}/nodes=1+{cluster='paravance'}/nodes=1+{c                     
         luster='paravance'}/nodes=1+{cluster='paravance'}/nodes=1+{clus                     
         ter='paravance'}/nodes=1+{cluster='paravance'}/nodes=1,walltime                     
         =00:40:00", 'command': 'sleep 31536000', 'queue': 'default'} on                     
         rennes                                                                              

...

Output()

storage restriction using memcached for  ['F']


Finished 2 tasks (apt,Start memcached with a pecifique config) on 
{'paravance-72.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

['172.16.96.20', '172.16.96.26', '172.16.96.30', '172.16.96.32', '172.16.96.44', '172.16.96.72']


In [6]:
if not config.execution_local:
    for i, machine in enumerate(config.machines):

        with config.enoslib.actions(roles=config.roles[machine["roles"][0]]) as p:
            #p.ensure_python()
            p.apt(name=["git","python3-pip"], state="present")
            p.command(
                task_name = "Delete the last version of the repo",
                cmd = "rm -rf /home/csimohammed/exp"
            )
            p.command(
                task_name="clone repo",
                cmd="git clone --single-branch --branch new_main https://github.com/cherifpy/traces-simulator.git /home/csimohammed/exp"
            )
            p.command(
                task_name = "installing python libs",
                cmd = "pip3 install pandas pylibmc numpy sockets PyYAML Flask pymemcache"
            )
        print(f"======= node {i} ========")
        
        if i==(NB_NODES-1):

            data = SendInfoToManager(i, CONFIG_GRAPHE,IPS_ADDRESS, REP_PORT)
            print(data)
            """with config.enoslib.actions(roles=config.roles[machine["roles"][0]]) as p:
                p.command(
                    task_name = "Executing the code on a site",
                    cmd = f"python3 /home/csimohammed/exp/cache-exp/replicaManager.py > /tmp/output{i}_and_errors.log 2>&1",
                    background=True
                )
            print(IPS_ADDRESS[i])
            sendObject(data, IPS_ADDRESS[i])"""
            

        else: 
            data = InfosToSend(i,CONFIG_GRAPHE, IPS_ADDRESS,REP_PORT,config.storage_capacities[i])
            with config.enoslib.actions(roles=config.roles[machine["roles"][0]]) as p:
                p.command(
                    task_name = "Executing the code on a site",
                    cmd = f"python3 /home/csimohammed/exp/cache-exp/cacheManager.py > /tmp/output{i}_and_errors.log 2>&1",
                    background=True
                )
            print(data)
            sendObject(data, IPS_ADDRESS[i])
            
        
        #config.enoslib.ensure_python3(True,roles=config.roles[machine["roles"][0]])
        
        infos_nodes.append({"node_ip":IPS_ADDRESS[i], "node_port":port_rep})
        port_rep += 1


Output()

Finished 4 tasks (apt,Delete the last version of the repo,clone repo,installing python libs) 
on {'paravance-20.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

Output()

======= node 0 ========


...

{'CACHE_SIZE': 536870912, 'infos': [{'latency': np.float64(20.0), 'id': 0, 'ip': '172.16.96.20', 'rep_port': 8880}, {'latency': np.float64(20.0), 'id': 1, 'ip': '172.16.96.26', 'rep_port': 8881}, {'latency': np.float64(20.0), 'id': 2, 'ip': '172.16.96.30', 'rep_port': 8882}, {'latency': np.float64(20.0), 'id': 3, 'ip': '172.16.96.32', 'rep_port': 8883}, {'latency': np.float64(60.0), 'id': 5, 'ip': '172.16.96.72', 'rep_port': 8885}], 'SITE_ID': 4, 'REP_PORT': 8884, 'IP_ADDRESS': '172.16.96.44', 'MANAGER_IP': '172.16.96.72'}


Output()

Finished 4 tasks (apt,Delete the last version of the repo,clone repo,installing python libs) 
on {'paravance-72.rennes.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

======= node 5 ========
{'IP_ADDRESS': '172.16.96.72', 'graphe_infos': array([[-1.,  1., 10., 20., 20., 60.],
       [-1., -1.,  8., 30., 20., 60.],
       [10.,  8., -1., 30., 20., 50.],
       [20., 30., -1., -1., 20., 31.],
       [20., 20., 20., 20., -1., 60.],
       [60., 60., 50., 31., 60., -1.]]), 'IPs_ADDRESS': ['172.16.96.20', '172.16.96.26', '172.16.96.30', '172.16.96.32', '172.16.96.44', '172.16.96.72'], 'infos': {0: {'latency': np.float64(60.0), 'id': 0, 'node_ip': '172.16.96.20', 'node_port': 8880}, 1: {'latency': np.float64(60.0), 'id': 1, 'node_ip': '172.16.96.26', 'node_port': 8881}, 2: {'latency': np.float64(50.0), 'id': 2, 'node_ip': '172.16.96.30', 'node_port': 8882}, 3: {'latency': np.float64(31.0), 'id': 3, 'node_ip': '172.16.96.32', 'node_port': 8883}, 4: {'latency': np.float64(60.0), 'id': 4, 'node_ip': '172.16.96.44', 'node_port': 8884}}}


In [5]:
#config.provider.destroy()